# PARSING PDF FILE TO EXTRACT INFORMATION ABOUT VARIABLES AND INSERT INTO METADATA TABLE

In [58]:
import pandas as pd

In [59]:
import os

In [60]:
import re

In [61]:
# pip install pdfminer.six
from pdfminer.high_level import extract_text

In [62]:
file_path = "../../GIT/demo-Turin-EuSilc/02_DataSet/" # Set the directory of the EU-SULC files
SQL_path = "./SQL_Folder/"

In [63]:
UDB_cIT23D = pd.read_csv(os.path.join(file_path,"UDB_cIT23D.csv"), sep= ",")
UDB_cIT23H = pd.read_csv(os.path.join(file_path,"UDB_cIT23H.csv"), sep= ",")
UDB_cIT23P = pd.read_csv(os.path.join(file_path,"UDB_cIT23P.csv"), sep= ",")
UDB_cIT23R = pd.read_csv(os.path.join(file_path,"UDB_cIT23R.csv"), sep= ",")

In [64]:
UDB_cIT23D.columns
    # UDB_cIT23H.columns
    # UDB_cIT23P.columns
    # UDB_cIT23R.columns

Index(['DB010', 'DB020', 'DB030', 'DB040', 'DB040_F', 'DB050', 'DB050_F',
       'DB060', 'DB060_F', 'DB062', 'DB062_F', 'DB070', 'DB070_F', 'DB075',
       'DB075_F', 'DB090', 'DB090_F', 'DB100', 'DB100_F', 'DB076', 'DB076_F'],
      dtype='object')

In [65]:
text = extract_text("Methodological_guidelines_2023.pdf")

In [ ]:
datasets = (UDB_cIT23D, UDB_cIT23H, UDB_cIT23P, UDB_cIT23R)
datasets_str = ("UDB_cIT23D", "UDB_cIT23H", "UDB_cIT23P", "UDB_cIT23R")
db_user = "C##EUSILC_SVIL" # Set with your user

output_file = os.path.join(SQL_path, "02_Insert_TD_UDB_cIT23_METADATA.sql")

def clean_text(text, max_length=3900):
    text = text.replace("'", " ")  # substitute , with " "
    text = text.replace(",", " ")  # substitute , with " "
    text = text.replace("\n", " ")  # substitute /n with " "
    text = re.sub(r"\s+", " ", text)  # reduces more spaces in only one
    return text.strip()[:max_length]  # remove initial and final spaces


for d in range(0,len(datasets)):

    column_vectors = datasets[d].columns

    with open(output_file, "a", encoding="utf-8") as file:
        file.write(f"\n--------------------------------------------------------------------\n")
        file.write(f"----------------------------- {datasets_str[d]} ---------------------------\n")
        file.write(f"--------------------------------------------------------------------\n\n")



    for i in range(0,len(column_vectors)):

        str1 = f"{column_vectors[i]}:"
        str2 = "Topic and detailed topic"
        str3 = "VALUES AND FORMAT"
        str4 = "FLAGS"
        str5 = "DESCRIPTION"

        # Regex to catch the 3 sections with limited characters between each string
        pattern = (
            rf"{re.escape(str1)}(.{{0,200}}?){re.escape(str2)}"  # des_variable
            rf".{{0,4000}}?"  
            rf"{re.escape(str3)}(.{{0,10000}}?){re.escape(str4)}"  # values_format
            rf".{{0,10000}}?"
            rf"{re.escape(str4)}(.{{0,10000}}?){re.escape(str5)}"  # flags
        )

        # find match in text
        match = re.search(pattern, text, re.DOTALL)

        # verify if the pattern is found
        if match:
            des_variable = clean_text(match.group(1))
            values_format = clean_text(match.group(2))
            flags = clean_text(match.group(3))


            with open(output_file, "a", encoding="utf-8") as file:
                file.write(f"INSERT INTO {db_user}.TD_UDB_cIT23_METADATA (COLUMN_NAME, COLUMN_DESCRIPTION, VALUES_AND_FORMAT, FLAGS)\nVALUES ('{column_vectors[i]}', '{des_variable}', '{values_format}', '{flags}');\n")

            if len(des_variable) > 128:
                print(f"Name Variable:\n{len(des_variable)} {column_vectors[i]}\n")

            if len(values_format) > 3000 | len(flags) > 3000:
                print(f"Values Format:\n{len(values_format)}\n")
                print(f"Flags:\n{len(flags)}\n")
        else:
            with open(output_file, "a", encoding="utf-8") as file:
                file.write(f"INSERT INTO {db_user}.TD_UDB_cIT23_METADATA (COLUMN_NAME, COLUMN_DESCRIPTION, VALUES_AND_FORMAT, FLAGS)\nVALUES ('{column_vectors[i]}', NULL, NULL, NULL);\n")


Values Format:
3900

Flags:
82

